<a href="https://colab.research.google.com/github/dinarasaurae/ai_assistant_investigations/blob/main/docsRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu sentence-transformers rank-bm25 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import json
import requests
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
import re
import time

In [ ]:
from huggingface_hub import login
login()

In [ ]:
LLM_API_URL = "http://94.126.205.209:9991/chat/"

In [ ]:
uploaded = files.upload()

Saving qna_dataset_smile_330_with_context.json to qna_dataset_smile_330_with_context.json


In [ ]:
uploaded = files.upload()

Saving smile_questions_with_context.json to smile_questions_with_context.json


In [ ]:
with open("smile_questions_with_context.json", "r", encoding="utf-8") as f:
    smile_questions = json.load(f)

with open("qna_dataset_smile_330_with_context.json", "r", encoding="utf-8") as f:
    smile_qna = json.load(f)

In [ ]:
vectorizer = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

чтобы всегда была корректная размерность

In [ ]:
vector_dim = vectorizer.get_sentence_embedding_dimension()

In [ ]:
vector_db = faiss.IndexFlatL2(vector_dim)

In [ ]:
questions = []
answers = []
contexts = []

In [ ]:
for item in smile_qna.values():
    contexts.append(item["Context"])
    questions.extend(item["questions_list"])
    answers.extend(item["answers_list"])

In [ ]:
def batch_extract_keywords(questions, batch_size=10, max_retries=3):
    processed = []
    session = requests.Session()

    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        batch_text = "\n".join([f"{idx+1}. {q}" for idx, q in enumerate(batch)])

        data = {
            "user_input": f"Выдели ключевые слова для следующих вопросов.\n"
                          f"Выводи строго в формате: '1. ключевые слова', '2. ключевые слова' и так далее.\n\n"
                          f"{batch_text}"
        }
        headers = {"Content-Type": "application/json"}

        retry_count = 0
        while retry_count < max_retries:
            try:
                response = session.post(LLM_API_URL, json=data, headers=headers, timeout=60)
                response.raise_for_status()
                batch_results = response.json().get("response", "").split("\n")

                # номера запросов не нужны
                cleaned_results = [res.split(". ", 1)[1].strip() if ". " in res else res.strip() for res in batch_results]
                processed.extend(cleaned_results)
                print(f"Батч {i // batch_size + 1}/{(len(questions) // batch_size) + 1} обработан успешно.")
                break

            except requests.exceptions.Timeout:
                retry_count += 1
                print(f"Таймаут (попытка {retry_count}/{max_retries}). Повтор через 5 сек...")
                time.sleep(5)

            except requests.exceptions.RequestException as e:
                print(f"Ошибка при запросе к LLM: {e}")
                processed.extend(batch)  # Если ошибка, оставляю ориг текст
                break

    session.close()
    return processed

затест

In [ ]:
response = requests.post(LLM_API_URL,
                         json={"user_input": "Какие ключевые слова у фразы 'Как авторизоваться?'"},
                         headers={"Content-Type": "application/json"},
                         timeout=120)
print(response.json())


{'response': 'Ключевыми словами в фразе "Как авторизоваться?" являются:\n1. Авторизация\n2. Как'}


Решила загрузить в processed.json только первые 30 вопросов, до тех пор, пока не настрою многопоточку в своём контейнер сервисе, или мне не дадут доступ к серверу

In [ ]:
def load_questions_from_json(json_path, limit=30):

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    questions = []
    for key in data:
        questions.extend(data[key]["questions_list"])
        if len(questions) >= limit:
            break

    return questions[:limit]

def batch_extract_keywords(questions, batch_size=10, max_retries=3):

    processed = []
    session = requests.Session()

    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        batch_text = "\n".join([f"{idx+1}. {q}" for idx, q in enumerate(batch)])

        data = {
            "user_input": f"Выдели ключевые слова для следующих вопросов.\n"
                          f"Выводи строго в формате: '1. ключевые слова', '2. ключевые слова' и так далее.\n\n"
                          f"{batch_text}"
        }
        headers = {"Content-Type": "application/json"}

        retry_count = 0
        while retry_count < max_retries:
            try:
                response = session.post(LLM_API_URL, json=data, headers=headers, timeout=60)
                response.raise_for_status()
                batch_results = response.json().get("response", "").split("\n")

                # номера вопросов лишние
                cleaned_results = [res.split(". ", 1)[1].strip() if ". " in res else res.strip() for res in batch_results]
                processed.extend(cleaned_results)
                print(f"Батч {i // batch_size + 1}/{(len(questions) // batch_size) + 1} обработан успешно.")
                break

            except requests.exceptions.Timeout:
                retry_count += 1
                print(f"Таймаут (попытка {retry_count}/{max_retries}). Повтор через 5 сек...")
                time.sleep(5)

            except requests.exceptions.RequestException as e:
                print(f"Ошибка при запросе к LLM: {e}")
                processed.extend(batch)
                break

    session.close()
    return processed

questions = load_questions_from_json("qna_dataset_smile_330_with_context.json", limit=30)

processed_keywords = batch_extract_keywords(questions, batch_size=1)

print("\n Пример обработанных вопросов:")
for idx, kw in enumerate(processed_keywords[:5]):
    print(f"{idx+1}. {kw}")

Батч 1/31 обработан успешно.
Батч 2/31 обработан успешно.
Батч 3/31 обработан успешно.
Батч 4/31 обработан успешно.
Батч 5/31 обработан успешно.
Батч 6/31 обработан успешно.
Батч 7/31 обработан успешно.
Батч 8/31 обработан успешно.
Таймаут (попытка 1/3). Повтор через 5 сек...
Батч 9/31 обработан успешно.
Батч 10/31 обработан успешно.
Батч 11/31 обработан успешно.
Батч 12/31 обработан успешно.
Батч 13/31 обработан успешно.
Батч 14/31 обработан успешно.
Батч 15/31 обработан успешно.
Батч 16/31 обработан успешно.
Батч 17/31 обработан успешно.
Батч 18/31 обработан успешно.
Батч 19/31 обработан успешно.
Батч 20/31 обработан успешно.
Батч 21/31 обработан успешно.
Батч 22/31 обработан успешно.
Батч 23/31 обработан успешно.
Батч 24/31 обработан успешно.
Батч 25/31 обработан успешно.
Батч 26/31 обработан успешно.
Батч 27/31 обработан успешно.
Батч 28/31 обработан успешно.
Батч 29/31 обработан успешно.
Батч 30/31 обработан успешно.

 Пример обработанных вопросов:
1. Авторизация, Платформа SMILE,

In [ ]:
output_data = {}
for idx, (question, keywords) in enumerate(zip(questions, processed_keywords)):
    output_data[idx] = {"question": question, "keywords": keywords}

with open("processed_keywords.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

print("Ключевые слова сохранены в processed_keywords.json!")

Ключевые слова сохранены в processed_keywords.json!


вот неудачная проба, потому что faiss и слишком малоинформативный промпт, и топовых результатов целых 50

(снова импортировала библиотеки, потому что коаб надоел прерыват сессии)

In [ ]:
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import rank_bm25

vectorizer = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

def load_questions_from_json(json_path):
    import json
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    questions = [q["question"] for q in data.values()]
    keywords = [q["keywords"] for q in data.values()]
    return questions, keywords

questions, keywords = load_questions_from_json("processed_keywords.json")

question_vectors = vectorizer.encode(questions, normalize_embeddings=True)
question_vectors = np.array(question_vectors, dtype=np.float32)

tokenized_questions = [q.lower().split() for q in questions]
bm25 = rank_bm25.BM25L(tokenized_questions)

SPECIFIC_TERMS = {"Яндекс", "Google", "Mail.ru", "VKontakte", "VK", "API"}

def contains_specific_terms(question):
    return any(term.lower() in question.lower() for term in SPECIFIC_TERMS)

LLM_API_URL = "http://94.126.205.209:9991/chat/"

def refine_query_with_llm(user_query):

    data = {"user_input": f"Сформулируй точный поисковый запрос для вопроса:\n{user_query}"}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(LLM_API_URL, json=data, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json().get("response", user_query)
    except requests.exceptions.RequestException:
        return user_query

def find_best_match(user_query):
    """
    BM25L+FAISS+LLM
    """
    refined_query = refine_query_with_llm(user_query)
    print(f"\n Уточненный запрос: {refined_query}")

    tokenized_query = refined_query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)

    penalty_scores = np.array([5 if contains_specific_terms(q) else 0 for q in questions])
    adjusted_scores = bm25_scores - penalty_scores  # попробовала снизить вес частных случаев

    top_bm25_idx = np.argsort(adjusted_scores)[-50:]

    top_bm25_idx = [idx for idx in top_bm25_idx if not contains_specific_terms(questions[idx])]

    if len(top_bm25_idx) < 10:
        top_bm25_idx = np.argsort(adjusted_scores)[-100:]
        top_bm25_idx = [idx for idx in top_bm25_idx if not contains_specific_terms(questions[idx])][:10]

    query_vector = vectorizer.encode([refined_query], normalize_embeddings=True).astype(np.float32)

    candidate_vectors = question_vectors[top_bm25_idx]
    index_temp = faiss.IndexFlatIP(candidate_vectors.shape[1])
    index_temp.add(candidate_vectors)

    D, I = index_temp.search(query_vector, 5)

    best_candidates = [questions[top_bm25_idx[i]] for i in I[0]]

    best_match = best_candidates[0]

    return best_match

user_input = "Как авторизоваться в системе через Яндекс?"
matched_question = find_best_match(user_input)

print(f"\n Найден ближайший вопрос: {matched_question}")



 Уточненный запрос: Как авторизоваться в системе через Яндекс?

 Найден ближайший вопрос: Какие данные необходимо ввести для авторизации после подтверждения почты?


Хороший вариант, выводит вопрос пользователя и ближайший вопрос из БД

In [ ]:
import requests
import numpy as np
import rank_bm25
from sentence_transformers import SentenceTransformer

def load_questions_from_json(json_path):
    import json
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    questions = [q["question"] for q in data.values()]
    keywords = [q["keywords"] for q in data.values()]
    return questions, keywords

questions, keywords = load_questions_from_json("processed_keywords.json")

tokenized_questions = [q.lower().split() for q in questions]
bm25 = rank_bm25.BM25Plus(tokenized_questions)

LLM_API_URL = "http://94.126.205.209:9991/chat/"

def refine_query_with_llm(user_query):

    data = {
        "user_input": (
            f"Ты ассистент по работе с документацией. "
            f"Твоя задача — превратить пользовательский вопрос в точный поисковый запрос. "
            f"Добавь ключевые слова, если это необходимо, но не изменяй смысл. "
            f"Формулируй так, чтобы запрос был удобен для поиска в документации.\n\n"
            f"Примеры:\n"
            f"1. Вопрос: 'Как авторизоваться в системе?' → Поисковый запрос: 'Методы авторизации в системе SMILE'\n"
            f"2. Вопрос: 'Как создать новый проект?' → Поисковый запрос: 'Инструкция по созданию проекта в SMILE'\n\n"
            f"Теперь обработай этот запрос:\n"
            f"Вопрос: '{user_query}'\n"
            f"Поисковый запрос:"
        )
    }

    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(LLM_API_URL, json=data, headers=headers, timeout=30)
        response.raise_for_status()
        refined_query = response.json().get("response", "").strip()

        if not refined_query or len(refined_query) < 5:
            return user_query

        return refined_query

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к LLM: {e}")
        return user_query

def rerank_candidates_with_llm(user_query, candidates):

    prompt = f"Какой из следующих вопросов наиболее похож на запрос: '{user_query}'?\n\n"
    prompt += "\n".join([f"{idx+1}. {q}" for idx, q in enumerate(candidates)])

    data = {"user_input": prompt}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(LLM_API_URL, json=data, headers=headers, timeout=30)
        response.raise_for_status()
        best_match_idx = int(response.json().get("response", "1").split(".")[0]) - 1
        return candidates[best_match_idx]
    except requests.exceptions.RequestException:
        return candidates[0]

def find_best_match(user_query):
    """
    BM25+LLM
    """
    refined_query = refine_query_with_llm(user_query)
    print(f"\n Уточненный запрос: {refined_query}")

    tokenized_query = refined_query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)

    top_bm25_idx = np.argsort(bm25_scores)[-10:]
    best_candidates = [questions[i] for i in reversed(top_bm25_idx)]

    best_match = rerank_candidates_with_llm(user_query, best_candidates)

    return best_match

user_input = "Как авторизоваться в системе через Яндекс?"
matched_question = find_best_match(user_input)

print(f"\n Найден ближайший вопрос: {matched_question}")



 Уточненный запрос: Поисковый запрос: 'Авторизация в системе через сервис Yandex'

 Найден ближайший вопрос: Какие действия необходимо выполнить для авторизации через социальную сеть Яндекс?


### __Собственно итоговый скрипт__

каждая строка в "question_list" из qna_dataset_smile_330_with_context.json соответствует "question" под id в processed_keywords.json

In [ ]:
import requests
import json
import numpy as np
import rank_bm25
from sentence_transformers import SentenceTransformer

def load_questions_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    questions = [q["question"] for q in data.values()]
    keywords = [q["keywords"] for q in data.values()]
    return questions, keywords

questions, keywords = load_questions_from_json("processed_keywords.json")

def load_qna_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

qna_data = load_qna_data("qna_dataset_smile_330_with_context.json")

tokenized_questions = [q.lower().split() for q in questions]
bm25 = rank_bm25.BM25Plus(tokenized_questions)

LLM_API_URL = "http://94.126.205.209:9991/chat/"

def refine_query_with_llm(user_query):
    """
    Улучшенное уточнение запроса с четкими инструкциями для LLM.
    """
    data = {
        "user_input": (
            f"Ты ассистент по работе с документацией. "
            f"Твоя задача — превратить пользовательский вопрос в точный поисковый запрос. "
            f"Добавь ключевые слова, если это необходимо, но не изменяй смысл. "
            f"Формулируй так, чтобы запрос был удобен для поиска в документации.\n\n"
            f"Примеры:\n"
            f"1. Вопрос: 'Как авторизоваться в системе?' → Поисковый запрос: 'Методы авторизации в системе SMILE'\n"
            f"2. Вопрос: 'Как создать новый проект?' → Поисковый запрос: 'Инструкция по созданию проекта в SMILE'\n\n"
            f"Теперь обработай этот запрос:\n"
            f"Вопрос: '{user_query}'\n"
            f"Поисковый запрос:"
        )
    }

    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(LLM_API_URL, json=data, headers=headers, timeout=30)
        response.raise_for_status()
        refined_query = response.json().get("response", "").strip()

        if not refined_query or len(refined_query) < 5:
            return user_query

        return refined_query

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к LLM: {e}")
        return user_query

def find_answer(matched_question):
    """
    Находит ответ на найденный вопрос в базе QnA.
    """
    for qna_id, qna_entry in qna_data.items():
        if matched_question in qna_entry["questions_list"]:
            index = qna_entry["questions_list"].index(matched_question)
            return qna_entry["answers_list"][index]  # ответ по индексу вопроса из спроцесшенного джейсона с ключевыми словами

    return "Ответ не найден в базе."

def find_best_match(user_query):
    """
    BM25+LLM
    """
    refined_query = refine_query_with_llm(user_query)
    print(f"\n Уточненный запрос: {refined_query}")

    tokenized_query = refined_query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)

    top_bm25_idx = np.argsort(bm25_scores)[-10:]  # топ-10
    best_candidates = [questions[i] for i in reversed(top_bm25_idx)]

    best_match = best_candidates[0]

    return best_match

user_input = "Как авторизоваться в системе через Яндекс?"
matched_question = find_best_match(user_input)

answer = find_answer(matched_question)

print(f"\n Найден ближайший вопрос: {matched_question}")
print(f"\n Ответ: {answer}")


 Уточненный запрос: Поисковый запрос: 'Авторизация в системе через сервис Yandex'

 Найден ближайший вопрос: Какие действия необходимо выполнить для авторизации через социальную сеть Яндекс?

 Ответ: Для авторизации через социальную сеть Яндекс необходимо нажать на кнопку Яндекс, которая находится на странице входа в SMILE. После этого вы попадете на платформу и сможете начать работу.


Изначально хотела обработать всё, но у меня ВМ CPU-only, по одному батчу это 100 минут, позже настрою брокер и прокси на сервис с LLM

In [ ]:
processed_questions = batch_extract_keywords(questions, batch_size=1)

Батч 1/331 обработан успешно.
Батч 2/331 обработан успешно.
Батч 3/331 обработан успешно.
Батч 4/331 обработан успешно.
Батч 5/331 обработан успешно.
Батч 6/331 обработан успешно.
Батч 7/331 обработан успешно.
Батч 8/331 обработан успешно.
Батч 9/331 обработан успешно.
Батч 10/331 обработан успешно.
Батч 11/331 обработан успешно.
Батч 12/331 обработан успешно.
Батч 13/331 обработан успешно.
Батч 14/331 обработан успешно.
Батч 15/331 обработан успешно.
Батч 16/331 обработан успешно.
Батч 17/331 обработан успешно.
Батч 18/331 обработан успешно.
Батч 19/331 обработан успешно.
Батч 20/331 обработан успешно.
Батч 21/331 обработан успешно.
Батч 22/331 обработан успешно.
Батч 23/331 обработан успешно.
Батч 24/331 обработан успешно.
Батч 25/331 обработан успешно.
Батч 26/331 обработан успешно.
Батч 27/331 обработан успешно.
Батч 28/331 обработан успешно.
Батч 29/331 обработан успешно.
Батч 30/331 обработан успешно.
Батч 31/331 обработан успешно.
Батч 32/331 обработан успешно.
Батч 33/331 обраб

KeyboardInterrupt: 